## QHDOPT for quadratic programming

In this notebook, we demonstrate how to employ QHDOPT to solve a quadratic programming problem with box constraints. 

Our target problem is $$\min \ f(x)=\frac{1}{2}x^TQx+b^Tx,$$ where $Q = \begin{bmatrix}-2 & 1 \\ 1 & -1 \end{bmatrix}, b = \begin{bmatrix}\frac{3}{4} \\ -\frac{1}{4}\end{bmatrix},$ and $x$ is a 2-dimensional variable vector with each entry constrained in $[0, 1]$.

We employ the QP mode of QHDOPT to input this problem.

### 1. Create problem instance

First, we import the class QHD from our package QHDOPT, implying the solver's algorithm. 

In [1]:
from qhdopt import QHD

/Users/samuelkushnir/miniconda3/envs/QHDOPT-dev/lib/python3.9/site-packages/qutip/__init__.py:65: UserWarning: The new version of Cython, (>= 3.0.0) is not supported.
  warnings.warn(


Next, we construct the matrices $Q$ and $b$ by Python lists. For the matrix $Q$, it is represented by a nested list. The vector $b$ is represented by a list, encoding its transposed matrix $b^T$. 

In [2]:
Q = [[-2, 1],
     [1, -1]]
bt = [3/4, -1/4]

Then we create a problem instance, stored in a variable `model`. It mandates the matrices $Q$ and $b$ to construct the problem, and by default set the box constraints to the unit box $[0, 1]^n$. You may override the bounds by `bounds=(l, r)` or `bounds=[(l1, r1), ..., (ln, rn)]`. 

In [3]:
model = QHD.QP(Q, bt)

### 2. Solve with D-Wave

Now we illustrate how to solve the problem with QHDOPT's solvers. We consider the D-Wave solver first. 

We can configure the D-Wave solver by running `model.dwave_setup` with all the parameters set. The mandatory parameter is the resolution $r$, which we set as 8. The API key can be either directly input by setting `api_key` or from a file. 

You may also set the annealing schedule, chain strength, embedding schemes, etc. Here we use default parameters. 

In [4]:
model.dwave_setup(resolution=8, api_key_from_file='')

To compile, send the job to run, and post-processing, you can run `model.optimize`. Setting `verbose=1` outputs more runtime information.

In [5]:
response = model.optimize(verbose = 1)

Backend QPU Time: 0.025723569999999998
Overhead Time: 2.49435437380188

* Runtime breakdown
SimuQ compilation: 0.000 s
Backend runtime: 2.520 s
Decoding time: 0.104 s
Fine-tuning time: 0.058 s
* Total time: 2.682 s
* Coarse solution
Unit Box Minimizer: [0. 1.]
Minimizer: [0. 1.]
Minimum: -0.75

* Fine-tuned solution
Minimizer: [0. 1.]
Minimum: -0.75
Unit Box Minimizer: [0. 1.]



Here, the coarse solution is one of the decoded solutions directly from D-Wave devices, fined-tuned solution is the best solution obtained by using classical local solvers to refine the coarse solutions, and the success rate is the portion of samples leading to the best solution. 

The D-Wave solver returns a global minimum at $x=\begin{bmatrix} 0 \\ 1 \end{bmatrix}$, with the minimum value $-0.75$. After fine-tuning, the minimum does not change in this case. 

A runtime breakdown is also provided to exhibit the time consumption of each step in the solver.

The Response object holds all relevant solution information in a structured way. It also contains more debugging and time information.

### 2. Compare with classical solvers

QHD is a quantum-classical hybrid algorithm, and we can compare its performance with classical-only solvers. QHDOPT contains a baseline backend where a random sampling procedure is followed by the specified post-processing method. Developers can also use it to debug the programmed `model`. 

In [4]:
response = model.classically_optimize(solver="IPOPT")


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In this minimal example, the classical solver performs well in the success rate and run time. In harder cases, QHDOPT with D-Wave backends normally performs better. 

### 3. Solve with QuTiP

The QHD algorithm can be deployed to different backends, thanks to SimuQ's Hamiltonian-based compilation scheme. Here we demonstrate how we can use a QuTiP-based solver to implement QHD and solve the QP problem. 

The workflow follows the same style. We first setup the QuTiP solver, then solve with `optimize`. 

In [6]:
model.qutip_setup(resolution=6, time_discretization=40)

In [7]:
model.optimize(verbose = 1)

Compiled.
Solved.
* Runtime breakdown
SimuQ compilation: 2.201 s
Backend runtime: 12.598 s
Decoding time: 0.025 s
Fine-tuning time: 0.078 s
* Total time: 14.903 s
* Coarse solution
Unit Box Minimizer: [0.16666667 1.        ]
Minimizer: [0.16666667 1.        ]
Minimum: -0.4861111044883728

* Fine-tuned solution
Minimizer: [0. 1.]
Minimum: -0.75
Unit Box Minimizer: [0. 1.]



### 4. Solve with IonQ

We can also solve the QP problem with IonQ backends. Similarly, we first setup the IonQ solver, then solve with `optimize`. 

In [8]:
model.ionq_setup(resolution=6, api_key_from_file='../ionq_API_key', time_discretization=10, shots = 1000, on_simulator=True)

In [9]:
model.optimize(verbose = 1)

{'id': 'a711150a-a932-42ad-bb6d-049d447b1421', 'status': 'ready', 'request': 1706081294}
* Coarse solution
Minimizer: [0.16666667 1.        ]
Minimum: -0.4861111040744517

* Fine-tuned solution
Minimizer: [0. 1.]
Minimum: -0.75
Success rate: 0.657

* Runtime breakdown
SimuQ compilation: 161.999 s
Backend runtime: 6.240 s
Decoding time: 0.054 s
Fine-tuning time: 2.914 s
* Total time: 171.206 s


-0.75

### 5. Obtain compilation details

For developers who need further details of the compilation procedure, we can print the intermediate parameters with the model. 

The D-Wave backend supports printing the hyper parameters and the final Hamiltonian. We may set `compile_only` for the `optimize` method to stop before sending the task to actual backends. 

In [10]:
model.dwave_setup(resolution=2, api_key_from_file='../dwave_API_key')
model.optimize(verbose=2, compile_only=True)

* Compilation information
Final Hamiltonian:
(Feature under development; only the Hamiltonian is meaningful here)
Quantum system:
- Sites: Q0 Q1 Q2 Q3 
- Sequentially evolves:
    Time = 1,  TIHamiltonian = -0.25  +  -0.046875 * Q1.Z  +  -0.078125 * Q0.Z  +  0.140625 * Q3.Z  +  -0.015625 * Q2.Z  +  0.0625 * Q1.Z * Q3.Z  +  0.0625 * Q1.Z * Q2.Z  +  0.0625 * Q0.Z * Q3.Z  +  0.0625 * Q0.Z * Q2.Z  +  -0.140625 * Q0.Z * Q1.Z  +  -0.140625 * Q2.Z * Q3.Z

Annealing schedule parameter: [[0, 0], [20, 1]]
Penalty coefficient: 0.140625
Chain strength: 0.375
Number of shots: 100


For QuTiP backend, QHDOPT can print the Hamiltonian. Notice that SimuQ stores quantum systems as piece-wise constant Hamiltonians, here we set the `time_discretization` to a small number. 

In [11]:
model.qutip_setup(resolution=2, time_discretization=3)
model.optimize(verbose=2, compile_only=True)

Compiled.
* Compilation information
Hamiltonian evolution:
Quantum system:
- Sites: Q0 Q1 Q2 Q3 
- Sequentially evolves:
    Time = 10,  TIHamiltonian = -1.0 * Q0.X * Q1.X  +  -1.0 * Q0.Y * Q1.Y  +  -1.0 * Q2.X * Q3.X  +  -1.0 * Q2.Y * Q3.Y
    Time = 0.5,  TIHamiltonian = -5.0 * Q0.X * Q1.X  +  -5.0 * Q0.Y * Q1.Y  +  -5.0 * Q2.X * Q3.X  +  -5.0 * Q2.Y * Q3.Y  +  -1.25 * Q1.Z  +  -1.25 * Q0.Z  +  -0.3125 * Q3.Z  +  0.3125 * Q1.Z * Q3.Z  +  0.625 * Q1.Z * Q2.Z  +  0.625 * Q0.Z * Q3.Z  +  1.25 * Q0.Z * Q2.Z
    Time = 0.5,  TIHamiltonian = -2.2222222222222223 * Q0.X * Q1.X  +  -2.2222222222222223 * Q0.Y * Q1.Y  +  -2.2222222222222223 * Q2.X * Q3.X  +  -2.2222222222222223 * Q2.Y * Q3.Y  +  -2.8125 * Q1.Z  +  -2.8125 * Q0.Z  +  -0.703125 * Q3.Z  +  0.703125 * Q1.Z * Q3.Z  +  1.40625 * Q1.Z * Q2.Z  +  1.40625 * Q0.Z * Q3.Z  +  2.8125 * Q0.Z * Q2.Z

Number of shots: 100


For IonQ backend, QHDOPT can print the Hamiltonian and the compiled circuit.

In [12]:
model.ionq_setup(resolution=2, api_key_from_file='../ionq_API_key', time_discretization=3, shots = 1000, on_simulator=True)
model.optimize(verbose=2, compile_only=True)

* Compilation information
Hamiltonian evolution:
Quantum system:
- Sites: Q0 Q1 Q2 Q3 
- Sequentially evolves:
    Time = 10,  TIHamiltonian = -1.0 * Q0.X * Q1.X  +  -1.0 * Q0.Y * Q1.Y  +  -1.0 * Q2.X * Q3.X  +  -1.0 * Q2.Y * Q3.Y
    Time = 0.5,  TIHamiltonian = -5.0 * Q0.X * Q1.X  +  -5.0 * Q0.Y * Q1.Y  +  -5.0 * Q2.X * Q3.X  +  -5.0 * Q2.Y * Q3.Y  +  -1.25 * Q1.Z  +  -1.25 * Q0.Z  +  -0.3125 * Q3.Z  +  0.3125 * Q1.Z * Q3.Z  +  0.625 * Q1.Z * Q2.Z  +  0.625 * Q0.Z * Q3.Z  +  1.25 * Q0.Z * Q2.Z
    Time = 0.5,  TIHamiltonian = -2.2222222222222223 * Q0.X * Q1.X  +  -2.2222222222222223 * Q0.Y * Q1.Y  +  -2.2222222222222223 * Q2.X * Q3.X  +  -2.2222222222222223 * Q2.Y * Q3.Y  +  -2.8125 * Q1.Z  +  -2.8125 * Q0.Z  +  -0.703125 * Q3.Z  +  0.703125 * Q1.Z * Q3.Z  +  1.40625 * Q1.Z * Q2.Z  +  1.40625 * Q0.Z * Q3.Z  +  2.8125 * Q0.Z * Q2.Z

Compiled circuit:
[{'gate': 'gpi', 'target': 0, 'phase': 0.0}, {'gate': 'ms', 'targets': [0, 1], 'phases': [0.0, 0.25], 'angle': 0.12500000000000003}, {'g